# Detailed run

An example of a complete run of lephare with all stages required to estimate redshift. In contrast to the two first notebooks we are not using the high level *prepare* and *process* methods. Instead we are using the more fundamental *filter*, *sedtolib*, *mag_gal*, and *zphota* which more resembles a command line based run.

We show how to include new filters from the Spanish Virtual Observatory [(SVO)](http://svo2.cab.inta-csic.es/theory/fps/).

Again this notebook uses the COSMOS2020 (Weaver et al. 2022) data as an example.

In [1]:
import os
import lephare as lp
import numpy as np
from matplotlib import pylab as plt

%matplotlib inline

LEPHAREDIR is being set to the default cache directory:
/home/cohen-tanugi/.cache/lephare/data
More than 1Gb may be written there.
LEPHAREWORK is being set to the default cache directory:
/home/cohen-tanugi/.cache/lephare/work
Default work cache is already linked. 
This is linked to the run directory:
/home/cohen-tanugi/.cache/lephare/runs/20241228T000922


## Set up the parameters


As for the previous notebooks we are starting with the default COSMOS config that ships with the lephare Python code.

Modification of three keywords of the parameter file. 

In [3]:
config = lp.default_cosmos_config.copy()

config.update(
    {
        # Verbose must be NO in the notebook.
        "VERBOSE": "NO",
        # this line reduced the zgrid density from the default to make the notebook run faster.
        # Comment this out for better science results
        "Z_STEP": "0.04,0.,6.",
    }
)

Then we get the auxiliary files required to run the notebook for the documentation. If you have cloned the full auxiliary data repository you do not need to run this.

In [4]:
lp.data_retrieval.get_auxiliary_data(
    keymap=config, additional_files=["examples/COSMOS.in", "examples/config.yml", "examples/output.para"]
)

Registry file downloaded and saved as data_registry.txt.
Checking/downloading 445 files...
445 completed.
All files downloaded successfully and are non-empty.
Checking/downloading 3 files...
3 completed.
All files downloaded successfully and are non-empty.


If adaption of the zero-points is turned off and you want to apply your own shift

In [ ]:
# config["AUTO_ADAPT"]= "NO"
# config["APPLY_SYSSHIFT"]= "0.049,-0.013,-0.055,-0.065,-0.042,-0.044,-0.065,-0.0156,-0.002,0.052,-0.006,0.071,
# 0.055,0.036,0.036,0.054,0.088,0.019,-0.154,0.040,0.044,0.060,0.045,0.022,0.062,0.033,0.015,0.012,0.0,0.0]"

## Create filter library

Read the filter names to be used in COSMOS.para and generate the filter file

First, you can use the standard method with the list of filters in the parameter file. The filters are store in the LEPHAREDIR/filt directory. You can pass either the config file or the keymap as argument

### Getting new filters

Each filter requires a filter response curve. This is a table of wavelength values in Angstrom and filter transmission in arbitrary units. In this example we get the filters we need from the [SVO](http://svo2.cab.inta-csic.es/theory/fps/). We could have also used the filters that are available in [$LEPHAREDIR/filt](https://github.com/lephare-photoz/lephare-data/tree/main/filt). Or one could also use local files.

In [5]:
# This would get the filters from the config file and local LEPHAREDIR/filt location.
# Later we see how to do the same from the SVO.
filterLib = lp.Filter(config_keymap=lp.all_types_to_keymap(config))
# uncomment to test passing the keymap
# filterLib = Filter(config_keymap=lp.all_types_to_keymap(config))
filterLib.run()

# NAME                        IDENT      Lbda_mean Lbeff(Vega)       FWHM     AB-cor    TG-cor      VEGA M_sun(AB)   CALIB      Lb_eff    Fac_corr
u_cfht.lowres                 1             0.3844      0.3908      0.0538    0.3150   -0.3891  -20.6345    6.0327       0      0.3815      1.0000
u_new.pb                      2             0.3690      0.3750      0.0456    0.6195   -0.2745  -20.8527    6.3135       0      0.3668      1.0000
gHSC.pb                       3             0.4851      0.4760      0.1194   -0.0860   -0.2458  -20.7272    5.0764       0      0.4780      1.0000
rHSC.pb                       4             0.6241      0.6142      0.1539    0.1466    0.2580  -21.5143    4.6523       0      0.6178      1.0000
iHSC.pb                       5             0.7716      0.7637      0.1476    0.3942    0.6138  -22.2286    4.5323       0      0.7666      1.0000
zHSC.pb                       6             0.8915      0.8907      0.0768    0.5169    0.7625  -22.6733    4.5147    

It is also possible to pass all the necessary keywords as arguments to the constructor:

In [13]:
#the config keymap from a config file can also be added to the constructor's arguments, in which case the keywords
#will be overridden by the explicit keywords passed as arguments below.
filterLib2 = lp.Filter(FILTER_REP="/home/cohen-tanugi/.cache/lephare/data/filt",
                       FILTER_LIST="cosmos/u_cfht.lowres,cosmos/u_new.pb,hsc/gHSC.pb,hsc/rHSC.pb,\
                       hsc/iHSC.pb,hsc/zHSC.pb,hsc/yHSC.pb,vista/Y.lowres,vista/J.lowres,vista/H.lowres,\
                       vista/K.lowres,cosmos/IB427.lowres,cosmos/IB464.lowres,cosmos/IB484.lowres,\
                       cosmos/IB505.lowres,cosmos/IB527.lowres,cosmos/IB574.lowres,cosmos/IB624.lowres,\
                       cosmos/IB679.lowres,cosmos/IB709.lowres,cosmos/IB738.lowres,cosmos/IB767.lowres,\
                       cosmos/IB827.lowres,cosmos/NB711.lowres,cosmos/NB816.lowres,vista/NB118.lowres,\
                       cosmos/irac_ch1.lowres,cosmos/irac_ch2.lowres,cosmos/irac_ch3.lowres,cosmos/irac_ch4.lowres",
                       TRANS_TYPE=1,
                       FILTER_CALIB="0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1",
                       FILTER_FILE="filter_cosmos"
                      )
filterLib2.run()

# NAME                        IDENT      Lbda_mean Lbeff(Vega)       FWHM     AB-cor    TG-cor      VEGA M_sun(AB)   CALIB      Lb_eff    Fac_corr
u_cfht.lowres                 1             0.3844      0.3908      0.0538    0.3150   -0.3891  -20.6345    6.0327       0      0.3815      1.0000
u_new.pb                      2             0.3690      0.3750      0.0456    0.6195   -0.2745  -20.8527    6.3135       0      0.3668      1.0000
gHSC.pb                       3             0.4851      0.4760      0.1194   -0.0860   -0.2458  -20.7272    5.0764       0      0.4780      1.0000
rHSC.pb                       4             0.6241      0.6142      0.1539    0.1466    0.2580  -21.5143    4.6523       0      0.6178      1.0000
iHSC.pb                       5             0.7716      0.7637      0.1476    0.3942    0.6138  -22.2286    4.5323       0      0.7666      1.0000
zHSC.pb                       6             0.8915      0.8907      0.0768    0.5169    0.7625  -22.6733    4.5147    

Second, you can use the filterSvc helper class to gain more freedom. This class allows to retrieve the list of filt objetcs, independently of writing them on file.

In [ ]:
filterLib = lp.FilterSvc.from_keymap(lp.all_types_to_keymap(config))
filter_output = os.path.join(os.environ["LEPHAREWORK"], "filt", config["FILTER_FILE"])
lp.write_output_filter(filter_output + ".dat", filter_output + ".doc", filterLib)

It also allows to load the filters from a yaml file, with the possibility to query the SVO service for filters

In [ ]:
!ls $LEPHAREDIR/examples/

In [ ]:
# We use an example yaml file to retrieve the filter names used by the SVO
filterLibSVO = lp.FilterSvc.from_yaml(f"{lp.LEPHAREDIR}/examples/config.yml")
filter_output = os.path.join(os.environ["LEPHAREWORK"], "filt", config["FILTER_FILE"])
lp.write_output_filter(filter_output + "_svo.dat", filter_output + "_svo.doc", filterLib)

Plot the filters. We can see slight differences between those on the SVO and in the lepahre database.

In [ ]:
fig = plt.figure(figsize=(15, 8))
for f, fsvo in zip(filterLib, filterLibSVO):
    d = f.data()
    plt.semilogx(d[0], d[1] / d[1].max())
    dsvo = fsvo.data()
    plt.semilogx(dsvo[0], dsvo[1] / dsvo[1].max(), ".")

In [ ]:
# filter_output = os.path.join(os.environ["LEPHAREWORK"],"filt", filterLib.keymap['FILTER_FILE'] + ".dat")
# This figure shows that the filters have differing normalisation which has no impact on the fitting process.
filters = np.loadtxt(
    filter_output + ".dat", dtype={"names": ("lamb", "val", "bid"), "formats": (float, float, int)}
)
plt.loglog(filters["lamb"], filters["val"])
plt.xlabel("wavelength");

## Create SED library

SED objects represent SED templates belonging to one of the three possible classes "STAR", "QSO" (for AGN type of objects), and "GAL" for galaxies. SED templates available with LePhare can be found under the `sed` directory. 

In [ ]:
sedlib = lp.Sedtolib(config_keymap=lp.all_types_to_keymap(config))

In [ ]:
sedlib.run(typ="STAR", star_sed="$LEPHAREDIR/sed/STAR/STAR_MOD_ALL.list")

In [ ]:
sedlib.run(typ="QSO", qso_sed="$LEPHAREDIR/sed/QSO/SALVATO09/AGN_MOD.list", gal_lib="LIB_QSO")

In [ ]:
sedlib.run(typ="GAL", gal_sed="$LEPHAREDIR/sed/GAL/COSMOS_SED/COSMOS_MOD.list", gal_lib="LIB_GAL")

## Create a magnitude library

Use the SED library to create a magnitude library

In [ ]:
maglib = lp.MagGal(config_keymap=lp.all_types_to_keymap(config))

In [ ]:
maglib.run(
    typ="STAR",
    lib_ascii="YES",
    star_lib_out="STAR_COSMOS",
    extinc_law="SB_calzetti.dat",
    mod_extinc="0,0",
)

In [ ]:
maglib.run(
    typ="QSO",
    lib_ascii="YES",
    mod_extinc="0,1000",
    eb_v="0.,0.1,0.2,0.3",
    extinc_law="SB_calzetti.dat",
    qso_lib_in="LIB_QSO",
    qso_lib_out="QSO_COSMOS",
)

In [ ]:
maglib.run(
    typ="GAL",
    lib_ascii="YES",
    gal_lib_in="LIB_GAL",
    gal_lib_out="GAL_COSMOS",
    mod_extinc="18,26,26,33,26,33,26,33",
    extinc_law="SMC_prevot.dat,SB_calzetti.dat,SB_calzetti_bump1.dat,SB_calzetti_bump2.dat",
    em_lines="EMP_UV",
    em_dispersion="0.5,0.75,1.,1.5,2.",
)

## Run the photoz

Read the parameter file and store the keywords. Example with the modification of three keywords of the parameter file. Verbose must be NO in the notebook.

In [ ]:
# These are the names created above with the argument gal_lib_out
config.update(
    {
        "ZPHOTLIB": "GAL_COSMOS,STAR_COSMOS,QSO_COSMOS",
        "SPEC_OUT": "save_spec",
    }
)

Instantiate a lephare.PhotoZ object which will manage the computation of photometric redshifts for all sources. It is instantiated based on all the config parameters. 



In [ ]:
photz = lp.PhotoZ(lp.all_types_to_keymap(config))

Read the input file with the following information: id, flux and associated uncertainties in all bands, a context indicating which bands to use in the fit (0 indicates all bands), and a spectrocopic redshift if it exists. 

In [ ]:
cat = np.loadtxt(f"{lp.LEPHAREDIR}/examples/COSMOS.in")
id = cat[:, 0]
fluxes = cat[:, 1:60:2]
efluxes = cat[:, 2:61:2]
context = cat[:, 61]
zspec = cat[:, 62]
print("Check format with context and zspec :", context, zspec)

Create a list of sources with a spec-z. Use for the zero-point training or any validation run.


In [ ]:
srclist = []
# Here, limited to the 1000 first sources.
n_obj = 1000
zspec_mask = np.logical_and(zspec > 0.01, zspec < 6)
# We are running on the last n_obj objects to use a different set of objects to perform
# zero-point correction to the test objects
for i in np.where(zspec_mask)[0][-n_obj:]:
    # Each element of the list is an instance of the lephare.onesource class.
    # This encapsulates all the information for a given source.
    oneObj = lp.onesource(i, photz.gridz)
    oneObj.readsource(str(id[i]), fluxes[i, :], efluxes[i, :], int(context[i]), zspec[i], " ")
    # lephare.PhotoZ passes the configuration parameters to each source.
    photz.prep_data(oneObj)
    srclist.append(oneObj)
print("Sources with a spec-z: ", len(srclist))

Derive the zero-points offsets. This corresponds to the median difference between apparent and observed magnitude in each filter. It is stored in the list, a0, which is later passed to the lephare.PhotoZ.run_photoz method. 

In [ ]:
a0 = photz.run_autoadapt(srclist)
offsets = ",".join(np.array(a0).astype(str))
offsets = "# Offsets from auto-adapt: " + offsets + "\n"
print(offsets)

Create the list of sources for which we want a photo-z. 

In [ ]:
photozlist = []
for i in range(n_obj):
    oneObj = lp.onesource(i, photz.gridz)
    oneObj.readsource(str(id[i]), fluxes[i, :], efluxes[i, :], int(context[i]), zspec[i], " ")
    photz.prep_data(oneObj)
    photozlist.append(oneObj)
print("Number of sources to be analysed: ", len(srclist))

Run the photoz. We pass the values of the zero point calibration calculated above

In [ ]:
photz.run_photoz(photozlist, a0)
# If adaption of the zero-points is turned off
# photz.run_photoz(photozlist[:100], [],[] )

### Save the parameters that have been used

For capturing the parameters that were used in a given run it is useful to save the updated config to file. Be careful as this will not capture the overrides that were sent directly to the lephare.MagGal.run method which impact the outputs.

In [ ]:
# we can write the config to a file to keep a record
lp.write_para_config(lp.all_types_to_keymap(config), "./config_file.para")
# One can also save it as a yaml file if you prefer
lp.write_yaml_config(lp.all_types_to_keymap(config), "./config_file.yaml")

## Create output in fits


In [ ]:
t = photz.build_output_tables(photozlist[:n_obj], para_out=None, filename="outputphotoz.fits")

In [ ]:
t[:5]

Create all ascii files with the output of the run

In [ ]:
import time

photz.write_outputs(photozlist[:10], int(time.time()))

In [ ]:
# This created the output ascii file specified in the config CAT_OUT parameter
!ls -al zphot.out

Check the results broadly follow a 1-1 relation

In [ ]:
plt.plot([0, 6], [0, 6], c="r", alpha=0.5)
plt.scatter(t["ZSPEC"], t["Z_BEST"], s=3)
plt.xlabel("z spec")
plt.ylabel("z phot")

Make plots for individual sources with all the files listed in save_spec

In [ ]:
from os import listdir
from os.path import isfile, join

listname = [f for f in listdir("save_spec") if isfile(join("save_spec", f))]
# Lets just look at the top 10
for namefile in listname[:10]:
    lp.plotspec("save_spec/" + str(namefile))